In [1]:
import pygame
from sys import exit
import numpy as np
import time
import math
import tensorflow as tf
import matplotlib.pyplot as plt
from guppy import hpy

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
def distance(x1,y1,x2,y2):
    return np.sqrt((x2 - x1)**2 + (y2 - y1)**2)

pygame.font.init() 
myfont = pygame.font.SysFont('Comic Sans MS', 60)

In [3]:
#Parameters for the game
acc = 0.1
max_speed = 7

In [4]:
class player:
    def __init__(self,x,y, color):
        self.x = x
        self.y = y
        self.color = color
        self.vx = 0
        self.vy = 0
        self.score = 0
        
    def draw(self):
        pygame.draw.circle(screen, self.color, (int(self.x), int(self.y)), 30)
   
    def update(self):
        if self.x < 30 or self.x > 770:
            self.vx = 0
            if self.x <30:
                self.x = 31
            else:
                self.x = 769
        if self.y < 30 or self.y > 570:
            self.vy = 0
            if self.y < 30:
                self.y = 31
            else:
                self.y = 569
        self.x = self.x + self.vx
        self.y = self.y + self.vy
        
class ball:
    def __init__(self):
        self.x = 400
        self.y = 295
        self.color = (240,240,240)
        self.vx = 0
        self.vy = 0
        
    def draw(self):
        pygame.draw.circle(screen, self.color, (int(self.x), int(self.y)), 15)
        
    def update(self):
        self.x = self.x + self.vx
        self.y = self.y + self.vy
        
         ####Check for goal
        if self.x < 15:
            if self.y > 200 and self.y <400:
                #player2.score += 1
                reset(200)
                return 2
        if self.x > 785:
            if self.y > 200 and self.y <400:
                #player1.score += 1
                reset(600)
                return 1
                
        if self.x < 15 or self.x > 785:
            self.vx = -self.vx
            if self.x <15:
                self.x = 16
            else:
                self.x = 784
        if self.y < 15 or self.y > 585:
            self.vy = -self.vy
            if self.y <15:
                self.y = 16
            else:
                self.y = 584
        self.vx = self.vx*0.9998
        self.vy = self.vy*0.9998
        l = distance(self.vx, self.vy, 0,0)
        if l > 0.00008:
            self.vx -=  self.vx/l*0.00008
            self.vy -=  self.vy/l*0.00008
        else:
            self.vx = 0
            self.vy = 0
        if math.isnan(self.x) or math.isnan(self.y):
            print("Ball position error")
            reset(0)
        return 0
    
def reset(a):
    player1.x = 100
    player1.y = np.random.randint(35, 565)
    player2.x = 700
    player2.y = np.random.randint(35, 565)
    ball0.x = 400#np.random.randint(200,600)
    ball0.y = np.random.randint(20, 580)
    player1.vx = 0
    player1.vy = 0
    player2.vx = 0
    player2.vy = 0
    ball0.vx = 0
    ball0.vy = 0

In [5]:
import tensorflow as tf
from tensorflow import keras
import os
os.environ["CUDA_VISIBLE_DEVICES"]="-1" 
#input parameter:
###Left, Right, ball.x, ball.y, player1.x, player1.y, player2.x, player2.y, ball.vx, ball.vy, player1.vx, player1.vy, player2.vx, player2.vy, player1.score, player2.score

#outout up, down, left, right, do nothing
class bot:
    def __init__(self):
        self.model = tf.keras.Sequential([
            tf.keras.layers.InputLayer(input_shape=(12,)),
            tf.keras.layers.Dense(20, activation="relu"),
            tf.keras.layers.Dense(5, activation="softmax")
        ])
        self.model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    
    def action(self, data):
        move = self.model.predict([data])
        return move[0].argsort()[0]

class Q:
    def __init__(self):
        self.model = tf.keras.Sequential([
            tf.keras.layers.InputLayer(input_shape=(12,)),
            tf.keras.layers.Dense(20, activation="sigmoid"),
            tf.keras.layers.Dense(1)
        ])
        self.model.compile(optimizer="adam", loss="MSE", metrics=["accuracy"])
    
    def value(self, data):
        pred = self.model.predict(tf.constant(data))
        return pred  
    
    def predict(self, data2red, data1red):
        ball0 = ball()
        data2 = [ball0.x/800.0 -  player2.x/800.0, ball0.y/600.0 -  player2.y/600.0, player2.x/800.0 - 1/2, player2.y/600.0 - 1/2, player1.x/800.0  - player2.x/800.0, player1.y/600.0 -  player2.y/600.0, ball0.vx/max_speed, ball0.vy/max_speed, player2.vx/max_speed, player2.vy/max_speed, player1.vx/max_speed, player1.vy/max_speed]
        data1 = [player1.x/800.0 - ball0.x/800.0, ball0.y/600.0 -  player1.y/600.0, 1/2 - player1.x/800.0, player1.y/600.0 - 1/2, player1.x/800.0  - player2.x/800.0, player2.y/600.0 -  player1.y/600.0, -ball0.vx/max_speed, ball0.vy/max_speed, -player1.vx/max_speed, player1.vy/max_speed, -player2.vx/max_speed, player2.vy/max_speed]
        #Transform data
        ballx_new = ball0.x + 5*ball0.vx
        ballvx_new = ball0.vx
        if ballx_new < 15:
            ballx_new = 30 - ballx_new
            ballvx_new = - ballvx_new
        if ballx_new > 785:
            ballx_new = 1570 - ballx_new
            ballvx_new = - ballvx_new

        bally_new = ball0.y + 5*ball0.vy
        ballvy_new = ball0.vy
        if bally_new < 15:
            bally_new = 30 - bally_new
            bballvy_new = - ballvy_new
        if bally_new > 785:
            bally_new = 1570 - bally_new
            ballvy_new = -ballvy_new

        player2x_new = [player2.x for i in range(4)]
        player2y_new = [player2.y for i in range(4)]
        player1x_new = [player1.x for i in range(4)]
        player1y_new = [player1.y for i in range(4)]
        player2vx_new = [1,0,-1,0]
        player1vx_new = [1,0,-1,0]
        player1vy_new = [0,-1,0,1]
        player2vy_new = [0,-1,0,1]

        player2x_new[0] += max_speed
        player1x_new[0] += max_speed
        if player2x_new[0] > 770:
            player2x_new[0] = 770 
        if player1x_new[0] > 770:
            player1x_new[0] = 770 

        player2x_new[2] -= max_speed
        player1x_new[2] -= max_speed
        if player2x_new[2] < 30:
            player2x_new[2] = 30
        if player1x_new[2] < 30:
            player1x_new[2] = 31 

        player2y_new[1] -= max_speed
        player1y_new[1] -= max_speed
        if player2x_new[1] < 30:
            player2x_new[1] = 30
        if player1x_new[1] < 30:
            player1x_new[1] = 31 

        player2y_new[3] -= max_speed
        player1y_new[3] -= max_speed
        if player2x_new[3] > 570:
            player2x_new[3] = 570
        if player1x_new[3] < 570:
            player1x_new[3] = 570 

        data_template = [data2, data2, data2, data2, data2, data1, data1, data1, data1, data1]
        for i in range(4):
            data_template[i] = [ballx_new/800.0 -  player2x_new[i] /800.0, bally_new/600.0 -  player2y_new[i]/600.0, player2x_new[i]/800.0 - 1/2, player2y_new[i]/600.0 - 1/2, player1.x /800.0  - player2x_new[i]/800.0, player1.y/600.0 -  player2y_new[i]/600.0, ballvx_new/max_speed, ballvy_new/max_speed, player2vx_new[i], player2vy_new[i], player1.vx/max_speed, player1.vy/max_speed]
            data_template[i+5] = [player1x_new[i]/800.0 - ballx_new/800.0, bally_new/600.0 -  player1y_new[i]/600.0, 1/2 - player1x_new[i]/800.0, player1y_new[i]/600.0 - 1/2, player1x_new[i]/800.0  - player2.x/800.0, player2.y/600.0 -  player1y_new[i]/600.0, -ballvx_new/max_speed, ballvy_new/max_speed, -player1vx_new[i], player1vy_new[i], -player2.vx/max_speed, player2.vy/max_speed]

        #outout up, down, left, right, do nothing

        a = self.value(data_template)
        pred2 = a[0:5]
        pred1 = a[5:10]
        return pred2, pred1
    
    
class bot_temp:
    def __init__(self, robot):
        self.robot = robot
        self.weights = [(robot.model.layers[i].get_weights())[0].tolist() for i in range(2)]
        for i in range(len(self.weights)):
            for j in range(len(self.weights[i])):
                self.weights[i][j] = np.random.normal(0, 1, len(self.weights[i][j]))
        self.fitness = 0
        self.games = 0
        
    def set_weights(self, weights):
        self.weights = weights
        return None
        
    def get_weights(self):
        return self.weights
    
    def update_bot(self):
        for i in range(len(self.weights)):
            temp = self.robot.model.layers[i].get_weights()
            temp[0] = np.array(self.weights[i])
            self.robot.model.layers[i].set_weights(temp)  
            
def vertical_crossover(weight0, weight1):
    weight = []
    for i in range(len(weight0)):
        a = np.random.randint(0,2)
        if a == 0:
            weight.append(weight0[i])
        else:
            weight.append(weight1[i])
    return weight

def average_crossover(weight0, weight1):
    weight = [weight0[i] for i in range(len(weight0))]
    for i in range(len(weight0)):
        for j in range(len(weight[i])):
            for k in range(len(weight[i][j])):
                weight[i][j][k] = (weight0[i][j][k] + weight1[i][j][k])/2.0
    return weight

def horizontal_crossover(weight0, weight1):
    weight = [weight0[i] for i in range(len(weight0))]
    for i in range(len(weight0)):
        for j in range(len(weight[i])):
            for k in range(len(weight[i][j])):
                weight[i][j][k] = (weight0[i][j][k] + weight1[i][j][k])/2.0
    return weight
              
def mutate(weight, n = 1, epsilon = 1):
    mutant = [weight[i] for i in range(len(weight))]
    for i in range(n):
        a = np.random.randint(0, len(weight))
        b = np.random.randint(0, len(weight[a]))
        c = np.random.randint(0, len(weight[a][b]))
        mutant[a][b][c] = weight[a][b][c] + np.random.normal(weight[a][b][c],epsilon*max(np.abs(weight[a][b][c]), 0.000001))
    return mutant

In [6]:
def save_weights(file, weights):
    temp = [weights[i] for i in range(len(weights))]
    data = []
    for i in range(len(temp)):
        for j in range(len(temp[i])):
            data.append(temp[i][j].tolist())
    temp = data
    data = []
    for i in range(len(temp)):
        for j in range(len(temp[i])):
            data.append(temp[i][j])      
    #print(np.shape(data))
    np.savetxt(file, data)
    return

def load_weights(file):
    data = np.loadtxt(file)
    loader = bot_temp(bot1)
    a = loader.get_weights()
    pos = 0
    for i in range(len(a)):
        for j in range(len(a[i])):
            for k in range(len(a[i][j])):
                    a[i][j][k] = data[pos]
                    pos += 1
    loader.set_weights(a)
    ret = loader.get_weights()
    del loader
    return ret

#for i in range(7):
    #file = line + "weights" + str(i) + ".csv"
    #population.append(bot_temp(bot1))
    #population[i].set_weights(load_weights(file))
    
def setup(pop_size, robot, line):
    population = [bot_temp(robot) for i in range(pop_size)]
    for c in range(len(population)):
        file =  '%sweights%s.csv' % (line,str(c))
        save_weights(file, population[c].get_weights())
    del population
    return

In [ ]:
#Q1 = Q()

player1 = player(100,300, (255,30,30))
player2 = player(700,300, (30,30,255))
ball0 = ball()
#ball0.y = 295
long_time = 0.2
#Convention 0 = Right, 1 = Up, 2 = Left, 3 = Down, 4 = Do nothing
pop_size = 100
line = ""
#setup(pop_size, bot1, line)

#fitness = [0 for i in range(pop_size)]
#games = [0 for i in range(pop_size)]
#inst1 = bot_temp(bot1)
#inst2 = bot_temp(bot1)

#best_score = []
#ave_score = []
goal_counter = 0

cutoff = 2500
generations = 1000
matches = 1
circle_time = time.time()
alpha = 0.8
gamma = 0.9

for g in range(generations):
    print("Generation", g)
    fitness1 = 0
    fitness2 = 0
    x = []
    y = []
    for mat in range(matches):
        for iteration in range(cutoff):
            t0 = time.time()
            if iteration%10 == 0:
                data2 = [ball0.x/800.0 -  player2.x/800.0, ball0.y/600.0 -  player2.y/600.0, player2.x/800.0 - 1/2, player2.y/600.0 - 1/2, player1.x/800.0  - player2.x/800.0, player1.y/600.0 -  player2.y/600.0, ball0.vx/max_speed, ball0.vy/max_speed, player2.vx/max_speed, player2.vy/max_speed, player1.vx/max_speed, player1.vy/max_speed]            
                data1 = [player1.x/800.0 - ball0.x/800.0, ball0.y/600.0 -  player1.y/600.0, 1/2 - player1.x/800.0, player1.y/600.0 - 1/2, player1.x/800.0  - player2.x/800.0, player2.y/600.0 -  player1.y/600.0, -ball0.vx/max_speed, ball0.vy/max_speed, -player1.vx/max_speed, player1.vy/max_speed, -player2.vx/max_speed, player2.vy/max_speed]
                pred2, pred1 = Q1.predict(data2, data1)
                tf.keras.backend.clear_session()
                action2 = np.argmax(pred2)
                action1 = np.argmax(pred1)
                if action1 % 2 == 1:
                    action1 = (action1 + 2) % 4
                r = np.random.binomial(1, max(1 - g/2500.0  - cutoff/6000.0,0))
                if r == 1:
                    a = 0
                    if player2.x > ball0.x:
                        a = 2
                    b = 3
                    if player2.y > ball0.y:
                        b = 1
                    c = 2
                    if player1.x > ball0.x:
                        c = 0
                    d = 3
                    if player2.y > ball0.y:
                        d = 1
                    action2 = np.random.choice([a,b])
                    action1 = np.random.choice([c,d])
                #fitness2 = fitness2/10.0
                #fitness1 = fitness1/10.0
                newQ2 = pred2[4] + alpha*(fitness2 + gamma*max(pred2) - pred2[4])
                newQ1 = pred1[4] + alpha*(fitness1 + gamma*max(pred1) - pred1[4])
                x.append(data2) 
                x.append(data1)
                y.append([newQ2])
                y.append([newQ1])
                fitness1 = 0
                fitness2 = 0
            if time.time() - t0 > long_time:
                print("Find Action2 takes long")
            t0 = time.time()
            #print(action2)
            if action2 == 4:
                #fitness[p2] -= 10/cutoff
                pass
            if action2 == 2:
                player2.vx -= acc
                if np.abs(player2.vx) > max_speed:
                    player2.vx = np.sign(player2.vx)*max_speed  
            elif player2.vx < 0:
                player2.vx += acc
            if action2 == 0:
                player2.vx += acc
                if np.abs(player2.vx) > max_speed:
                        player2.vx = np.sign(player2.vx)*max_speed  
            elif player2.vx > 0:
                player2.vx -= acc
            if action2 != 2  and action2 != 0:
                #player2.vx = 0
                pass

            if action2 == 3:
                player2.vy += acc
                if np.abs(player2.vy) > max_speed:
                    player2.vy = np.sign(player2.vy)*max_speed    
            elif player2.vy > 0:
                player2.vy -= acc
            if action2 == 1:
                player2.vy -= acc
                if np.abs(player2.vy) > max_speed:
                    player2.vy = np.sign(player2.vy) *max_speed  
            elif player2.vy < 0:
                player2.vy += acc
            if action2 != 3 and action2 != 1:
                #player2.vy = 0
                pass
            if time.time() - t0 > long_time:
                print("Take Action2 takes long")
            t0 = time.time()

            #print(action1)
            if action1 == 4:
                #fitness[p2] -= 10/cutoff
                pass

            if action1 == 0:
                player1.vx -= acc
                if np.abs(player1.vx) > max_speed:
                    player1.vx = np.sign(player1.vx)*max_speed  
            elif player1.vx < 0:
                player1.vx += acc        

            if action1 == 2:
                player1.vx += acc
                if np.abs(player1.vx) > max_speed:
                    player1.vx = np.sign(player1.vx)*max_speed 
            elif player1.vx > 0:
                player1.vx -= acc
            if action1 != 2 and action1 != 0:
                #player1.vx = 0 
                pass

            if action1 == 3:
                player1.vy += acc
                if np.abs(player1.vy) > max_speed:
                    player1.vy = np.sign(player1.vy)*max_speed    
            elif player1.vy > 0:
                player1.vy -= acc
            if action1 == 1:
                player1.vy -= acc
                if np.abs(player1.vy) > max_speed:
                    player1.vy = np.sign(player1.vy) *max_speed 
            elif player1.vy < 0:
                player1.vy += acc
            if action1 != 1 and action1 != 3:
                #player1.vy = 0
                pass
            if time.time() - t0 > long_time:
                print("Take Action1 takes long")
            t0 = time.time()

            ####Collision Check
            dp1b = distance(player1.x, player1.y, ball0.x, ball0.y)
            if math.isnan(dp1b):
                print("Distance error 1") 
                print(player1.x, player1.y, ball0.x, ball0.y)
                break
            else:
                fitness1 -= (dp1b - 45) /(cutoff*5.0)
                fitness1 -= (distance(800, 300, ball0.x, ball0.y) - 15) /(cutoff*4.0)
                pass
            if dp1b < 45:
                fitness1 += 10
                nx = ball0.x - player1.x
                ny = ball0.y - player1.y
                if nx == 0 and ny == 0:
                    ny += 0.1
                l = distance(nx, ny, 0,0)*1.0
                proj = (nx*ball0.vx + ny*ball0.vy)/l
                proj2 = (nx*player1.vx + ny*player1.vy)/l
                ball0.vx =  ball0.vx - 2*proj*nx/l + proj2*nx/l
                ball0.vy =  ball0.vy - 2*proj*ny/l + proj2*ny/l
                player1.vx = 0
                player1.vy = 0
                ball0.x = player1.x + nx/l * (45 + int(acc + 1))
                ball0.y = player1.y + ny/l * (45 + int(acc + 1))

            dp2b = distance(player2.x, player2.y, ball0.x, ball0.y)
            if math.isnan(dp2b):
                print("Distance error 2")
                print(player2.x, player2.y, ball0.x, ball0.y)
                break
            else:
                fitness2 -= (dp2b - 45) /cutoff*5.0
                fitness2 -= (distance(0, 300, ball0.x, ball0.y) - 15) /cutoff*4.0
                pass
            if dp2b < 45:
                fitness2 += 10
                nx = ball0.x - player2.x
                ny = ball0.y - player2.y
                if nx == 0 and ny == 0:
                    ny += 0.1
                l = distance(nx, ny, 0,0)
                proj = (nx*ball0.vx + ny*ball0.vy)/l
                proj2 = (nx*player2.vx + ny*player2.vy)/l
                ball0.vx =  ball0.vx - 2*proj*nx/l + proj2*nx/l
                ball0.vy =  ball0.vy - 2*proj*ny/l + proj2*ny/l
                player2.vx = 0
                player2.vy = 0
                ball0.x = player2.x + nx/l * (45 + int(acc + 1))
                ball0.y = player2.y + ny/l * (45 + int(acc + 1))

            if distance(player2.x, player2.y, player1.x, player1.y) < 60:
                nx = player2.x - player1.x
                ny = player2.y - player1.y
                if nx == 0 and ny == 0:
                    nx = 2
                    player1.x -= 1
                    player2.x += 1
                mx = (player2.x + player1.x)/2.0
                my = (player2.y + player1.y)/2.0
                l = distance(player2.x, player2.y, player1.x, player1.y)
                player2.x = mx + nx/l*30
                player2.y = my + ny/l*30
                player1.x = mx - nx/l*30
                player1.y = my - ny/l*30

            if distance(player1.x, player1.y, ball0.x, ball0.y) < 35 or distance(player2.x, player2.y, ball0.x, ball0.y) < 35:
                ball0.x = 400
                ball0.y = 16
                ball0.vx = 0
                ball0.vy = 0
                #pygame.time.delay(500)
            if time.time() - t0 > long_time:
                print("Collision Check takes long")
            t0 = time.time()


            ####Update everything
            a = ball0.update()
            if a == 1:
                fitness1 += 1000.0/cutoff*1000
                fitness2 -= 1000.0/cutoff*150
                player1.score += 1
                goal_counter += 1
            if a == 2:
                fitness2 += 1000.0/cutoff*1000
                fitness1 -= 1000.0/cutoff*150
                player2.score += 1
                goal_counter += 1
            player1.update()
            player2.update()
            if time.time() - t0 > long_time:
                print("Update takes long")
            t0 = time.time()
    Q1.model.fit(tf.constant(x),tf.constant(y), epochs = 20, verbose = 0)
    print(time.time()-circle_time, "Goals scored:", goal_counter)
    circle_time = time.time()
    goal_counter = 0

Generation 0
9.763973236083984 Goals scored: 0
Generation 1
Find Action2 takes long
Find Action2 takes long
9.413940668106079 Goals scored: 0
Generation 2
Find Action2 takes long
Find Action2 takes long
8.45805311203003 Goals scored: 1
Generation 3
Find Action2 takes long
7.963318347930908 Goals scored: 0
Generation 4
Find Action2 takes long
8.642470598220825 Goals scored: 1
Generation 5
Find Action2 takes long
9.934621572494507 Goals scored: 1
Generation 6
8.596274375915527 Goals scored: 1
Generation 7
Find Action2 takes long
8.778003215789795 Goals scored: 0
Generation 8
Find Action2 takes long
9.304895639419556 Goals scored: 1
Generation 9
7.528920888900757 Goals scored: 0
Generation 10
7.9309751987457275 Goals scored: 0
Generation 11
Find Action2 takes long
8.393335819244385 Goals scored: 1
Generation 12
7.629409551620483 Goals scored: 1
Generation 13
Find Action2 takes long
9.389123678207397 Goals scored: 1
Generation 14
7.958442449569702 Goals scored: 0
Generation 15
8.2280762195

Find Action2 takes long
Find Action2 takes long
38.90993905067444 Goals scored: 1
Generation 140
8.032168626785278 Goals scored: 0
Generation 141
8.03481125831604 Goals scored: 0
Generation 142
Find Action2 takes long
838.0261442661285 Goals scored: 0
Generation 143
8.880642414093018 Goals scored: 0
Generation 144
9.0685715675354 Goals scored: 1
Generation 145
Find Action2 takes long
9.781269073486328 Goals scored: 0
Generation 146
8.34073781967163 Goals scored: 0
Generation 147
8.78696084022522 Goals scored: 0
Generation 148
8.689131736755371 Goals scored: 1
Generation 149
9.586433172225952 Goals scored: 0
Generation 150
9.770607233047485 Goals scored: 0
Generation 151
8.382486820220947 Goals scored: 0
Generation 152
9.118451595306396 Goals scored: 0
Generation 153
10.905812501907349 Goals scored: 1
Generation 154
10.977922677993774 Goals scored: 1
Generation 155
8.372579574584961 Goals scored: 1
Generation 156
8.874299764633179 Goals scored: 0
Generation 157
7.9790825843811035 Goals 

In [ ]:
plt.plot(best_score)
plt.plot(ave_score)
plt.show()

In [ ]:
h=hpy()
h.heap()

In [ ]:
#Initialize the pygame
#pygame.init()

#create the screen
screen = pygame.display.set_mode((800,600))

#Title and Icon
pygame.display.set_caption("Soccer")



running = True

player1 = player(100,300, (255,30,30))
player2 = player(700,300, (30,30,255))
ball = ball()

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        keys = pygame.key.get_pressed()
    acc = 0.007
    max_speed = 1
    if keys[pygame.K_LEFT]:
        player2.vx -= acc
        if np.abs(player2.vx) > max_speed:
            player2.vx = np.sign(player2.vx)*max_speed  
    elif player2.vx < 0:
        player2.vx += acc
    if keys[pygame.K_RIGHT]:
        player2.vx += acc
        if np.abs(player2.vx) > max_speed:
                player2.vx = np.sign(player2.vx)*max_speed  
    elif player2.vx > 0:
        player2.vx -= acc
    if not keys[pygame.K_LEFT] and not keys[pygame.K_RIGHT]:
        #player2.vx = 0
        pass
    
    if keys[pygame.K_DOWN]:
        player2.vy += acc
        if np.abs(player2.vy) > max_speed:
            player2.vy = np.sign(player2.vy)*max_speed    
    elif player2.vy > 0:
        player2.vy -= acc
    if keys[pygame.K_UP]:
        player2.vy -= acc
        if np.abs(player2.vy) > max_speed:
            player2.vy = np.sign(player2.vy) *max_speed  
    elif player2.vy < 0:
        player2.vy += acc
    if not keys[pygame.K_DOWN] and not keys[pygame.K_UP]:
        #player2.vy = 0
        pass
                
    if keys[pygame.K_a]:
        player1.vx -= acc
        if np.abs(player1.vx) > max_speed:
            player1.vx = np.sign(player1.vx)*max_speed  
    elif player1.vx < 0:
        player1.vx += acc        
    
    if keys[pygame.K_d]:
        player1.vx += acc
        if np.abs(player1.vx) > max_speed:
            player1.vx = np.sign(player1.vx)*max_speed 
    elif player1.vx > 0:
        player1.vx -= acc
    if not keys[pygame.K_a] and not keys[pygame.K_d]:
        #player1.vx = 0 
        pass
    
    if keys[pygame.K_s]:
        player1.vy += acc
        if np.abs(player1.vy) > max_speed:
            player1.vy = np.sign(player1.vy)*max_speed    
    elif player1.vy > 0:
        player1.vy -= acc
    if keys[pygame.K_w]:
        player1.vy -= acc
        if np.abs(player1.vy) > max_speed:
            player1.vy = np.sign(player1.vy) *max_speed 
    elif player1.vy < 0:
        player1.vy += acc
    if not keys[pygame.K_s] and not keys[pygame.K_w]:
        #player1.vy = 0
        pass

    ####Collision Check
    if distance(player1.x, player1.y, ball.x, ball.y) < 45:
        nx = ball.x - player1.x
        ny = ball.y - player1.y
        l = distance(nx, ny, 0,0)
        proj = (nx*ball.vx + ny*ball.vy)/l
        proj2 = (nx*player1.vx + ny*player1.vy)/l
        ball.vx =  ball.vx - 2*proj*nx/l + proj2*nx/l
        ball.vy =  ball.vy - 2*proj*ny/l + proj2*ny/l
        player1.vx = 0
        player1.vy = 0
        ball.x = player1.x + nx/l *47
        ball.y = player1.y + ny/l *47
        
    if distance(player2.x, player2.y, ball.x, ball.y) < 45:
        nx = ball.x - player2.x
        ny = ball.y - player2.y
        l = distance(nx, ny, 0,0)
        proj = (nx*ball.vx + ny*ball.vy)/l
        proj2 = (nx*player2.vx + ny*player2.vy)/l
        ball.vx =  ball.vx - 2*proj*nx/l + proj2*nx/l
        ball.vy =  ball.vy - 2*proj*ny/l + proj2*ny/l
        player2.vx = 0
        player2.vy = 0
        ball.x = player2.x + nx/l *47
        ball.y = player2.y + ny/l *47
    
    if distance(player2.x, player2.y, player1.x, player1.y) < 60:
        nx = player2.x - player1.x
        ny = player2.y - player1.y
        mx = (player2.x + player1.x)/2.0
        my = (player2.y + player1.y)/2.0
        l = distance(player2.x, player2.y, player1.x, player1.y)
        player2.x = mx + nx/l*30
        player2.y = my + ny/l*30
        player1.x = mx - nx/l*30
        player1.y = my - ny/l*30
        
    if distance(player1.x, player1.y, ball.x, ball.y) < 35 or distance(player2.x, player2.y, ball.x, ball.y) < 35:
        ball.x = 400
        ball.y = 16
        ball.vx = 0
        ball.vy = 0
        #pygame.time.delay(500)
        
        
    
    ####Update everything
    ball.update()
    player1.update()
    player2.update()
    
     
    ####Draw everyting
        #The field
    screen.fill((10,150,10))
   
    pygame.draw.line(screen,(255,255,255),(798,200),(798,400))
    pygame.draw.line(screen,(255,255,255),(2,200),(2,400))
    pygame.draw.line(screen,(255,255,255),(400,0),(400,600))
    text_surface = myfont.render(str(player1.score), True, (0, 0, 0))
    screen.blit(text_surface, (730, 10))
    text_surface = myfont.render(str(player2.score), True, (0, 0, 0))
    screen.blit(text_surface, (40, 10))
    
        #The players
    player1.draw()
    player2.draw()
    
        #The ball
    ball.draw()
    pygame.display.update()
pygame.display.quit()
pygame.quit()

In [ ]:
import gc

#Initialize the pygame
pygame.init()

#create the screen
screen = pygame.display.set_mode((800,600))

#Title and Icon
pygame.display.set_caption("Soccer")

bot1 = bot()   

inst1 = bot_temp(bot1)
inst2 = bot_temp(bot1)
inst1.set_weights(load_weights('weights7.csv'))
inst2.set_weights(load_weights('weights0.csv'))

running = True

player1 = player(100,300, (255,30,30))
player2 = player(700,300, (30,30,255))
ball0 = ball()
reset(100)
for v in range(20000):
    if running == False:
        break
    for j in range(1):
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            keys = pygame.key.get_pressed()
        for k in range(1): 
            for l in range(1):
                time1 = time.time()
                t0 = time.time()
                if v % 5 == 0:
                    data2 = [ball0.x/800.0 -  player2.x/800.0, ball0.y/600.0 -  player2.y/600.0, player2.x/800.0 - 1/2, player2.y/600.0 - 1/2, player1.x/800.0  - player2.x/800.0, player1.y/600.0 -  player2.y/600.0, ball0.vx/max_speed, ball0.vy/max_speed, player2.vx/max_speed, player2.vy/max_speed, player1.vx/max_speed, player1.vy/max_speed]            
                    data1 = [player1.x/800.0 - ball0.x/800.0, ball0.y/600.0 -  player1.y/600.0, 1/2 - player1.x/800.0, player1.y/600.0 - 1/2, player1.x/800.0  - player2.x/800.0, player2.y/600.0 -  player1.y/600.0, -ball0.vx/max_speed, ball0.vy/max_speed, -player1.vx/max_speed, player1.vy/max_speed, -player2.vx/max_speed, player2.vy/max_speed]
                    pred2, pred1 = Q1.predict(data2, data1)
                    action2 = np.argmax(pred2)
                    action1 = np.argmax(pred1)
                    r = np.random.binomial(1, max(1 - ((v)/3000.0),0))
                    if r == 1:
                        a = 0
                        if player2.x > ball0.x:
                            a = 2
                        b = 3
                        if player2.y > ball0.y:
                            b = 1
                        c = 2
                        if player1.x > ball0.x:
                            c = 0
                        d = 3
                        if player2.y > ball0.y:
                            d = 1
                        action2 = np.random.choice([a,b])
                        action1 = np.random.choice([c,d])
                if time.time() - t0 > long_time:
                    print("Find Action2 takes long")
                t0 = time.time()
                #print(action2)
                if action2 == 4:
                    #fitness[p2] -= 10/cutoff
                    pass
                if action2 == 2:
                    player2.vx -= acc
                    if np.abs(player2.vx) > max_speed:
                        player2.vx = np.sign(player2.vx)*max_speed  
                elif player2.vx < 0:
                    player2.vx += acc
                if action2 == 0:
                    player2.vx += acc
                    if np.abs(player2.vx) > max_speed:
                            player2.vx = np.sign(player2.vx)*max_speed  
                elif player2.vx > 0:
                    player2.vx -= acc
                if action2 != 2  and action2 != 0:
                    #player2.vx = 0
                    pass

                if action2 == 3:
                    player2.vy += acc
                    if np.abs(player2.vy) > max_speed:
                        player2.vy = np.sign(player2.vy)*max_speed    
                elif player2.vy > 0:
                    player2.vy -= acc
                if action2 == 1:
                    player2.vy -= acc
                    if np.abs(player2.vy) > max_speed:
                        player2.vy = np.sign(player2.vy) *max_speed  
                elif player2.vy < 0:
                    player2.vy += acc
                if action2 != 3 and action2 != 1:
                    #player2.vy = 0
                    pass
                if time.time() - t0 > long_time:
                    print("Find Action1 takes long")
                t0 = time.time()
                #print(action1)
                if action1 == 4:
                    pass

                if action1 == 0:
                    player1.vx -= acc
                    if np.abs(player1.vx) > max_speed:
                        player1.vx = np.sign(player1.vx)*max_speed  
                elif player1.vx < 0:
                    player1.vx += acc        

                if action1 == 2:
                    player1.vx += acc
                    if np.abs(player1.vx) > max_speed:
                        player1.vx = np.sign(player1.vx)*max_speed 
                elif player1.vx > 0:
                    player1.vx -= acc
                if action1 != 2 and action1 != 0:
                    #player1.vx = 0 
                    pass

                if action1 == 3:
                    player1.vy += acc
                    if np.abs(player1.vy) > max_speed:
                        player1.vy = np.sign(player1.vy)*max_speed    
                elif player1.vy > 0:
                    player1.vy -= acc
                if action1 == 1:
                    player1.vy -= acc
                    if np.abs(player1.vy) > max_speed:
                        player1.vy = np.sign(player1.vy) *max_speed 
                elif player1.vy < 0:
                    player1.vy += acc
                if action1 != 1 and action1 != 3:
                    #player1.vy = 0
                    pass
                if time.time() - t0 > long_time:
                    print("Take Action1 takes long")
                t0 = time.time()
                
                ####Collision Check
                dp1b = distance(player1.x, player1.y, ball0.x, ball0.y)
                if math.isnan(dp1b):
                    print("Distance error 1") 
                    print(player1.x, player1.y, ball0.x, ball0.y)
                    break
              
                if dp1b < 45:
                    #fitness[p1] += 5
                    nx = ball0.x - player1.x
                    ny = ball0.y - player1.y
                    if nx == 0 and ny == 0:
                        ny += 0.1
                    l = distance(nx, ny, 0,0)*1.0
                    proj = (nx*ball0.vx + ny*ball0.vy)/l
                    proj2 = (nx*player1.vx + ny*player1.vy)/l
                    ball0.vx =  ball0.vx - 2*proj*nx/l + proj2*nx/l
                    ball0.vy =  ball0.vy - 2*proj*ny/l + proj2*ny/l
                    player1.vx = 0
                    player1.vy = 0
                    ball0.x = player1.x + nx/l * (45 + int(acc + 1))
                    ball0.y = player1.y + ny/l * (45 + int(acc + 1))
                
                dp2b = distance(player2.x, player2.y, ball0.x, ball0.y)
                if math.isnan(dp2b):
                    print("Distance error 2")
                    print(player2.x, player2.y, ball0.x, ball0.y)
                    break

                if dp2b < 45:
                    #fitness[p2] += 5
                    nx = ball0.x - player2.x
                    ny = ball0.y - player2.y
                    if nx == 0 and ny == 0:
                        ny += 0.1
                    l = distance(nx, ny, 0,0)
                    proj = (nx*ball0.vx + ny*ball0.vy)/l
                    proj2 = (nx*player2.vx + ny*player2.vy)/l
                    ball0.vx =  ball0.vx - 2*proj*nx/l + proj2*nx/l
                    ball0.vy =  ball0.vy - 2*proj*ny/l + proj2*ny/l
                    player2.vx = 0
                    player2.vy = 0
                    ball0.x = player2.x + nx/l * (45 + int(acc + 1))
                    ball0.y = player2.y + ny/l * (45 + int(acc + 1))

                if distance(player2.x, player2.y, player1.x, player1.y) < 60:
                    nx = player2.x - player1.x
                    ny = player2.y - player1.y
                    if nx == 0 and ny == 0:
                        nx = 2
                        player1.x -= 1
                        player2.x += 1
                    mx = (player2.x + player1.x)/2.0
                    my = (player2.y + player1.y)/2.0
                    l = distance(player2.x, player2.y, player1.x, player1.y)
                    player2.x = mx + nx/l*30.0
                    player2.y = my + ny/l*30.0
                    player1.x = mx - nx/l*30.0
                    player1.y = my - ny/l*30.0

                if distance(player1.x, player1.y, ball0.x, ball0.y) < 35 or distance(player2.x, player2.y, ball0.x, ball0.y) < 35:
                    reset()
                    #pygame.time.delay(500)
                if time.time() - t0 > long_time:
                    print("Collision Check takes long")
                t0 = time.time()
        
    tf.keras.backend.clear_session()
    ####Update everything
    a = ball0.update()
    player1.update()
    player2.update()
    if a == 1:
        player1.score += 1
    if a == 2:
        player2.score += 1
     
    ####Draw everyting
        #The field
    screen.fill((10,150,10))
   
    pygame.draw.line(screen,(255,255,255),(798,200),(798,400))
    pygame.draw.line(screen,(255,255,255),(2,200),(2,400))
    pygame.draw.line(screen,(255,255,255),(400,0),(400,600))
    text_surface = myfont.render(str(player1.score), True, (0, 0, 0))
    screen.blit(text_surface, (40, 10))
    text_surface = myfont.render(str(player2.score), True, (0, 0, 0))
    screen.blit(text_surface, (730, 10))
    
        #The players
    player1.draw()
    player2.draw()
    
        #The ball
    ball0.draw()
    pygame.display.update()

pygame.display.quit()
#pygame.quit()

In [ ]:
data = np.sav('data.npy')

In [ ]:
name = 'bot' + str(1)
print(name)

In [ ]:
print(rk)
fitness = [fitness[i] for i in rk]
print(fitness)

In [ ]:
gc.collect()

In [ ]:
file =  '%sweights%s.csv' % (str(1),str(2))
print(file)

In [ ]:
a =[1]
a.append([1,2])
print(a)

In [ ]:
!pip install objgrap

In [ ]:
print()